# Solving a Sudoku puzzle with z3


In [19]:
!pip install z3-solver
from z3 import *

Create an Integer Variable for each cell of the Sudoku grid:

In [20]:
k = 5
k2 = k ** 2

In [21]:
X = [[Int(f"x_{i + 1}_{j + 1}") for j in range(k2)] for i in range(k2)]

Requirement 1: each cell must contain a digit (1 to k2)

In [22]:
cells_c  = [And(X[i][j] >= 1, X[i][j] <= k2) for i in range(k2) for j in range(k2) ]

Requirement 2: Every digit has to be placed exactly once in each row:

In [23]:
rows_c   = [ Distinct([ X[i][j0] 
                        for i in range(k2)]) 
            for j0 in range(k2) ]

Requirement 3: Every digit has to be placed exactly once in each row:

In [24]:
cols_c   = [ Distinct([ X[i0][j] 
                        for j in range(k2)]) 
             for i0 in range(k2)]

Requirement 4: Every digit has to be placed exactly once in each $3 \times 3$ subgrid

In [25]:
sq_c     = [ Distinct([ X[k*i0 + i][k*j0 + j] 
                        for i in range(k) for j in range(k) ]) 
             for i0 in range(k) for j0 in range(k) ]

Now, put all the requirements together

In [26]:
sudoku_c = cells_c + rows_c + cols_c + sq_c

and consider some concrete puzzle:

In [27]:
instance = [[0] * k2] * k2


In [28]:
instance

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

and push the filled spaces and a requirement:

In [29]:
instance_c = [ If(instance[i][j] == 0, 
                  True, 
                  X[i][j] == instance[i][j]) 
               for i in range(16) for j in range(16) ]

Build a z3 specification: 

In [30]:
s = Solver()                                            # (1)
s.add(sudoku_c + instance_c)                            # (2)


Check its satisfability:

In [31]:
s.check()

sat

Print the solution (if sat):

In [32]:
if s.check() == sat:                                    # (3)
    m = s.model()                                       # (4)
    r = [ [ m.evaluate(X[i][j]) for j in range(k2) ]     # (5)
          for i in range(k2) ]
    print_matrix(r[0])                                     # (6)
else:
    print("failed to solve")        

[21,
 10,
 25,
 23,
 19,
 13,
 7,
 11,
 14,
 20,
 9,
 8,
 5,
 12,
 3,
 1,
 17,
 18,
 16,
 2,
 6,
 4,
 24,
 15,
 22]


# Exercise

Prepare your z3 model to solve generic sudokus, i.e. puzzles with sizes $k^2 \times k^2$, where $k$ is the number of different digits involved.



Then, apply it to solve the following puzzles of sizes $2^2 \times 2^2$, $3^2 \times 3^2$ and $4^2 \times 4^2$


In [33]:
instance2 = ((1,0,0,0),
             (0,2,0,0),
             (0,0,3,0),
             (0,0,0,4))

In [34]:
instance3 = ((5,3,0,0,7,0,0,0,0),
            (6,0,0,1,k2,5,0,0,0),
            (0,k2,8,0,0,0,0,6,0),
            (8,0,0,0,6,0,0,0,3),
            (4,0,0,8,0,3,0,0,1),
            (7,0,0,0,2,0,0,0,6),
            (0,6,0,0,0,0,2,8,0),
            (0,0,0,4,1,k2,0,0,5),
            (0,0,0,0,8,0,0,7,k2))

In [35]:
instance4=((15,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0))